# sample_weightを掛けて動作確認

## (1) テストデータ／環境準備

<b><a href="01.ipynb">こちらのレポート</a></b>ですでに作成済みのモデル／ベクトライザーを使用します。

In [1]:
'''
    プロトタイピング用のパスと、Botライブラリーパスを取得／設定します
'''
import sys
import os

prototype_dir = os.path.join(os.getcwd(), '..')
prototype_dir = os.path.abspath(prototype_dir)

learning_dir = os.path.join(prototype_dir, '..')
learning_dir = os.path.abspath(learning_dir)
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

print('prototype_dir=%s\nlearning_dir=%s' % (prototype_dir, learning_dir))

prototype_dir=/Users/makmorit/GitHub/donusagi-bot/learning/prototype
learning_dir=/Users/makmorit/GitHub/donusagi-bot/learning


In [2]:
'''
    初期設定
    データファイル、エンコードを指定
    内容は、learn.py を参考にしました。    
'''
from learning.core.learn.learning_parameter import LearningParameter
bot_id = 8888
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': None,
    # 'algorithm': LearningParameter.ALGORITHM_NAIVE_BAYES
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    # 'params_for_algorithm': { 'C': 200 }
    'params_for_algorithm': {}
}
learning_parameter = LearningParameter(attr)

## (2) 負例データの仕込み

proba < 0.5 で、かつ意図した回答にならない質問文を任意に拾い上げます。

In [3]:
'''
    Reply クラスを生成し予測実行
'''
from prototype.modules.ReplyForLocalTest import Reply
reply = Reply(bot_id, learning_parameter)

test_X = [
    'サーバとインフラの料金はいくらかかりますか？', # -->3673「以下URLに接続しxxxをご確認ください」
    '再編集されたメールを検索したいのですが？', # -->3402「方法については『xxx(送信済メールの再利用)』を参照してください」
    '本文をインデントする方法が知りたいのですが？', # -->3488「URLのパスを””、<>で囲っていただくことで、改善されるか確認ください。」    
    '未読メールがあるはずなのに見つかりません。', # -->3940「方法については『xxx(未読のメールのフォルダを作成する)』を参照してください」
    '迷惑メールを受け取りたくないのですが？', # -->3697「一般的には、「広告メール」のことをいいます」
]
z = reply.predict(test_X)

TextArray#__init__ start
2017/03/06 PM 07:14:04 TextArray#__init__ start
TextArray#to_vec start
2017/03/06 PM 07:14:04 TextArray#to_vec start
TextArray#to_vec end
2017/03/06 PM 07:14:04 TextArray#to_vec end
predicted results (order by probability desc)
2017/03/06 PM 07:14:04 predicted results (order by probability desc)
{'probability': 0.69139462087656989, 'answer_id': 3673.0}
2017/03/06 PM 07:14:04 {'probability': 0.69139462087656989, 'answer_id': 3673.0}
{'probability': 0.1899476457383148, 'answer_id': 3950.0}
2017/03/06 PM 07:14:04 {'probability': 0.1899476457383148, 'answer_id': 3950.0}
{'probability': 0.013690085497197156, 'answer_id': 3799.0}
2017/03/06 PM 07:14:04 {'probability': 0.013690085497197156, 'answer_id': 3799.0}
{'probability': 0.012180466458260522, 'answer_id': 3502.0}
2017/03/06 PM 07:14:04 {'probability': 0.012180466458260522, 'answer_id': 3502.0}
{'probability': 0.010356139530454285, 'answer_id': 3408.0}
2017/03/06 PM 07:14:04 {'probability': 0.010356139530454285, 

question: サーバとインフラの料金はいくらかかりますか？
answer: 3673
proba: 0.691394620877 

question: 再編集されたメールを検索したいのですが？
answer: 3402
proba: 0.688213759181 

question: 本文をインデントする方法が知りたいのですが？
answer: 3488
proba: 0.571904988964 

question: 未読メールがあるはずなのに見つかりません。
answer: 3940
proba: 0.545927508699 

question: 迷惑メールを受け取りたくないのですが？
answer: 3697
proba: 0.971456449706 



## (3) 負例データを訓練データに追加

上記の誤回答を誘発するデータを「反面教師データ」として、訓練データに追加します。

sample_weight が訓練データ１レコード（CSV１行分）ごとに設定される重み係数であるための措置です。

In [4]:
csv_file_name = 'test_daikin_conversation.csv'
added_csv_file_name = 'test_daikin_conversation_ext.csv'

csv_file_path = os.path.join(prototype_dir, 'resources', csv_file_name)
appended_csv_file_path = os.path.join(prototype_dir, 'resources', added_csv_file_name)
appended_csv_file_path

'/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_daikin_conversation_ext.csv'

In [5]:
import difflib
import sys

with open(csv_file_path, 'r') as old_csv:
    with open(appended_csv_file_path, 'r') as new_csv:
        diff = difflib.unified_diff(
            old_csv.readlines(),
            new_csv.readlines(),
            fromfile='original',
            tofile='added',
        )
        for line in diff:
            sys.stdout.write(line)

--- original
+++ added
@@ -89859,3 +89859,8 @@
 DKI企画部インフラG ＜net-unyo@daikin.co.jp＞"
 1403260,VPNキーの再発行依頼を行ったが、進捗状況を教えて欲しい。,4672,"VPNキー再発行は1週間を目処に対応しているため、申請から1週間を経過している場合や緊急事態の場合は、以下へお電話ください。
 7-205-413"
+1403261,サーバとインフラの料金はいくらかかりますか？,3673,"負例データ"
+1403262,再編集されたメールを検索したいのですが？,3402,"負例データ"
+1403263,本文をインデントする方法が知りたいのですが？,3488,"負例データ"
+1403264,未読メールがあるはずなのに見つかりません。,3940,"負例データ"
+1403265,迷惑メールを受け取りたくないのですが？,3697,"負例データ"


## (4) 負例データを含む訓練データで学習実行

bot_id は誤回答を誘発する「反面教師データ」を含まないモデル（8888）とは別にしておきます（比較のため）。

In [6]:
'''
    Bot/Reply モジュールをカスタマイズした
    BotForLocalTest/ReplyForLocalTest モジュールは、
    {prototype_dir}/modules 配下に格納されています
    （ローカル環境から MySQLdb/dataset に接続できないための措置）
'''
from prototype.modules.BotForLocalTest import Bot

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


#### sample_weight は、グリッドサーチの fit 関数にはパラメータ指定ができません。

別段制約事項として注記はないのですが、ご参考までに、グリッドサーチの fit 関数の仕様は下記URLになります。

http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html#sklearn.grid_search.GridSearchCV.fit

したがって、コストパラメータ（L2 penalty）は即値指定とします（C=1.0）。

C=1.0 とした根拠は、sample_weight（X, y, Cの逆数と一緒にソルバー関数に与えられる引数、後述）を効かせるための措置になります。

下記URLを参考にしました。

http://scikit-learn.org/0.17/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn-linear-model-logisticregression

In [7]:
'''
    初期設定
    データファイル、エンコードを指定
    内容は、learn.py を参考にしました。    
'''
appended_bot_id = 9999
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': None,
    # 'algorithm': LearningParameter.ALGORITHM_NAIVE_BAYES
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    'params_for_algorithm': { 'C': 1.0 } # <---グリッドサーチが使えないので、即値を指定。
    # 'params_for_algorithm': {}
}
appended_learning_parameter = LearningParameter(attr)

#### 負例データに関する sample_weight を仮に 0.3 と設定します。

In [8]:
'''
    sample_weight の生成
    負例データを 0.3、その他はデフォルトの 1.0 にします。
    負例データ＝５件、その他のデータ＝17,443件、計＝17,448件
'''
import numpy as np
sample_weight = np.full(17448, 1.0)
for i in range(17443, 17448):
    sample_weight[i] = 0.3
sample_weight[-8:] # <---配列の値を確認します。最後尾の５件のウェイトが 0.3 です

array([ 1. ,  1. ,  1. ,  0.3,  0.3,  0.3,  0.3,  0.3])

#### 負例データ＋sample_weightによる学習を実行します。

In [9]:
'''
    Bot クラスを生成し学習実行
'''
bot = Bot(appended_bot_id, appended_learning_parameter)
evaluator = bot.learn(csv_file_path=appended_csv_file_path, 
                      csv_file_encoding='utf-8', 
                      sample_weight=sample_weight,
                      solver='lbfgs')

learning_parameter: {'_include_tag_vector': False, '_include_failed_data': False, '_algorithm': 0, '_classify_threshold': None, '_params_for_algorithm': {'C': 1.0}}
2017/03/06 PM 07:14:05 learning_parameter: {'_include_tag_vector': False, '_include_failed_data': False, '_algorithm': 0, '_classify_threshold': None, '_params_for_algorithm': {'C': 1.0}}
start Bot#learn
2017/03/06 PM 07:14:05 start Bot#learn
TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 17448
2017/03/06 PM 07:14:05 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 17448
TextArray#__init__ start
2017/03/06 PM 07:14:05 TextArray#__init__ start
TextArray#to_vec start
2017/03/06 PM 07:14:05 TextArray#to_vec start
TextArray#to_vec end
2017/03/06 PM 07:14:24 TextArray#to_vec end
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ..., 

#### 補足：ソルバー指定について

LogisticRegression の引数に与える solver=‘lbfgs’ とした根拠は、上記URLの

- For multiclass problems, only ‘newton-cg’ and ‘lbfgs’ handle

- ‘newton-cg’, ‘lbfgs’ and ‘sag’ only handle L2 penalty.


の記述に則った措置になります。

 すなわちペナルティを効かせてマルチクラスの予測がしたかったら、newton-cg、lbfgs しか選択肢がないと言っているようです.

 たしかにソースを参照したところ、X、y、C の逆数値とともに、sample_weight が solver 関数の引数に与えられているのを確認しました.
 
 （下記は<b><a href="https://github.com/scikit-learn/scikit-learn/blob/51a765a/sklearn/linear_model/logistic.py">ソース</a></b>からの抜粋です）

~~~
         elif solver == 'newton-cg':
            args = (X, target, 1. / C, sample_weight)
            w0, n_iter_i = newton_cg(hess, func, grad, w0, args=args,
                                     maxiter=max_iter, tol=tol)
~~~


solver に対応すると思われる手法については、下記URLを参考にしました。

http://www.kamishima.net/mlmpyja/lr/optimization.html

もっとも・・・‘newton-cg’ と ‘lbfgs’ の違いが 2017/03/06 時点で体感的に分かりづらかったため、処理が軽い（＝Newton-CG と比してへシアンが不要のこと）と判断した lbfgs を指定した次第です。

## (5) ご参考：負例データ＋sample_weight無し時の学習結果との比較

#### （r2_score の比較は、後日行います。しばらくお待ち願います）

負例データ無し時：（検証時のデータは<b><a "href=01.ipynb">こちらになります</a></b>）
~~~~
accuracy: 0.984177940839
~~~~

負例データ有り時：Accuracyが落ちてますが、正直、こんなに落ちるとは・・・といった感じです
~~~~
accuracy: 0.931224209078
~~~~

## (6) 動作確認：予測実行

誤回答を誘発する「反面教師データ」からの質問に対しては、いずれも proba がしきい値の 0.5 を下回っている、という効果が見られました。

In [10]:
'''
    Reply クラスを生成し予測実行
'''
appended_reply = Reply(appended_bot_id, appended_learning_parameter)

test_X = [
    'サーバとインフラの料金はいくらかかりますか？', # -->3673「以下URLに接続しxxxをご確認ください」
    '再編集されたメールを検索したいのですが？', # -->3402「方法については『xxx(送信済メールの再利用)』を参照してください」
    '本文をインデントする方法が知りたいのですが？', # -->3488「URLのパスを””、<>で囲っていただくことで、改善されるか確認ください。」    
    '未読メールがあるはずなのに見つかりません。', # -->3940「方法については『xxx(未読のメールのフォルダを作成する)』を参照してください」
    '迷惑メールを受け取りたくないのですが？', # -->3697「一般的には、「広告メール」のことをいいます」
]
appended_z = appended_reply.predict(test_X)

TextArray#__init__ start
2017/03/06 PM 07:25:17 TextArray#__init__ start
TextArray#to_vec start
2017/03/06 PM 07:25:17 TextArray#to_vec start
TextArray#to_vec end
2017/03/06 PM 07:25:17 TextArray#to_vec end
predicted results (order by probability desc)
2017/03/06 PM 07:25:17 predicted results (order by probability desc)
{'probability': 0.022099641719661892, 'answer_id': 3799.0}
2017/03/06 PM 07:25:17 {'probability': 0.022099641719661892, 'answer_id': 3799.0}
{'probability': 0.020839153785434682, 'answer_id': 3673.0}
2017/03/06 PM 07:25:17 {'probability': 0.020839153785434682, 'answer_id': 3673.0}
{'probability': 0.018065903854403625, 'answer_id': 3950.0}
2017/03/06 PM 07:25:17 {'probability': 0.018065903854403625, 'answer_id': 3950.0}
{'probability': 0.013831039690359032, 'answer_id': 3663.0}
2017/03/06 PM 07:25:17 {'probability': 0.013831039690359032, 'answer_id': 3663.0}
{'probability': 0.013216024377531313, 'answer_id': 3684.0}
2017/03/06 PM 07:25:17 {'probability': 0.01321602437753

question: サーバとインフラの料金はいくらかかりますか？
answer: 3799
proba: 0.0220996417197 

question: 再編集されたメールを検索したいのですが？
answer: 3402
proba: 0.0356665088787 

question: 本文をインデントする方法が知りたいのですが？
answer: 3488
proba: 0.0410813703997 

question: 未読メールがあるはずなのに見つかりません。
answer: 3940
proba: 0.20406655081 

question: 迷惑メールを受け取りたくないのですが？
answer: 3697
proba: 0.320363263338 



他方、誤回答を誘発しない「正規の教師データ」からの質問に対しても、proba がかなり低くなっています。

<b>これは明らかな不具合なのですが、現時点で原因が不明ですので、後日調査いたします。</b>

In [17]:
'''
    誤回答を誘発しない質問文を与えて、動作に影響が及んでいないかの確認。
'''
test_X_prop = [
    'Outlook2010にて、メールの署名の作り方が教えてください。', # -->3398
    'アウトルック2010で、メーリングリストの差出人のみに返信したい', # -->3460
]
appended_z = appended_reply.predict(test_X_prop)

TextArray#__init__ start
2017/03/06 PM 07:35:15 TextArray#__init__ start
TextArray#to_vec start
2017/03/06 PM 07:35:15 TextArray#to_vec start
TextArray#to_vec end
2017/03/06 PM 07:35:15 TextArray#to_vec end
predicted results (order by probability desc)
2017/03/06 PM 07:35:15 predicted results (order by probability desc)
{'probability': 0.57693813863389531, 'answer_id': 3398.0}
2017/03/06 PM 07:35:15 {'probability': 0.57693813863389531, 'answer_id': 3398.0}
{'probability': 0.018654437722176079, 'answer_id': 3409.0}
2017/03/06 PM 07:35:15 {'probability': 0.018654437722176079, 'answer_id': 3409.0}
{'probability': 0.012143664646494929, 'answer_id': 3442.0}
2017/03/06 PM 07:35:15 {'probability': 0.012143664646494929, 'answer_id': 3442.0}
{'probability': 0.010561930671814301, 'answer_id': 3940.0}
2017/03/06 PM 07:35:15 {'probability': 0.010561930671814301, 'answer_id': 3940.0}
{'probability': 0.0091388151676327061, 'answer_id': 3461.0}
2017/03/06 PM 07:35:15 {'probability': 0.009138815167632

question: Outlook2010にて、メールの署名の作り方が教えてください。
answer: 3398
proba: 0.576938138634 

question: アウトルック2010で、メーリングリストの差出人のみに返信したい
answer: 3460
proba: 0.472297180459 

